In [1]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import copy
from torch import optim

from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.datasets import fetch_california_housing

import yaml

/usr/local/miniconda3/envs/nine/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from model import main

In [3]:
import argparse
parser = argparse.ArgumentParser(description = '')
parser.add_argument('-n', '--experiment_name', type = str, default = '', help = 'experiment name')
parser.add_argument('--alpha', type = float, default = 0.95, help = 'splitting probability alpha')
parser.add_argument('--beta', type = float, default = 2., help = 'splitting probability beta')
parser.add_argument('--var_height', type = float, default = 1, help = 'variance for height prior')
parser.add_argument('--b_hyperparam1', type = float, default = -1, help = 'hyperparameter1 for b prior')
parser.add_argument('--b_hyperparam2', type = float, default = 1, help = 'hyperparameter2 for b prior')
parser.add_argument('--gamma_shape', type = float, default = 1, help = 'hyperparameter1 for gamma prior')
parser.add_argument('--gamma_scale', type = float, default = 1, help = 'hyperparameter2 for gamma prior')
parser.add_argument('--grow_prob', type = float, default = 0.4, help = 'GROW probability')
parser.add_argument('--prune_prob', type = float, default = 0.4, help = 'PRUNE probability')
parser.add_argument('--change_prob', type = float, default = 0.2, help = 'CHANGE probability')
args = parser.parse_args([])

In [4]:
data = fetch_california_housing(as_frame=True)
data_x, data_y = data["data"] , data["target"]
columns_list = data["feature_names"]

data_x = np.array(data_x)
data_y = np.array(data_y).reshape(-1,1)

In [5]:
with open('model/config.yaml', 'r') as f:
    config = yaml.safe_load(f)
for k, v in vars(args).items():
    if v is not None and k in config:
        config[k] = v
    
config['var_height'] = 0.01
config['max_depth'] = data_x.shape[1]

#For Test
config["num_samples"] = 10

In [6]:
random_state = 0
train_x,test_x,train_y,test_y = train_test_split(data_x,data_y, test_size=0.3, random_state=random_state)

scaler = StandardScaler()
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

y_mean = np.mean(train_y)
y_std = np.std(train_y)

In [7]:
samples = main.main(train_x, train_y.flatten(), config, test_x, test_y.flatten() , y_mean , y_std)

/home/psh/ANOVA-BART-newversion-official/model/btree.py:99: RuntimeWarning: overflow encountered in exp
  term4 = np.exp(term4_2 - term4_1)


0th sample train rmse : 0.7443, test rmse : 0.7551, # ensemble : 200
1th sample train rmse : 0.6758, test rmse : 0.6873, # ensemble : 200
2th sample train rmse : 0.6403, test rmse : 0.6576, # ensemble : 200
3th sample train rmse : 0.6241, test rmse : 0.6418, # ensemble : 200
4th sample train rmse : 0.6037, test rmse : 0.6215, # ensemble : 200
5th sample train rmse : 0.5934, test rmse : 0.6139, # ensemble : 200
6th sample train rmse : 0.5861, test rmse : 0.6073, # ensemble : 200
7th sample train rmse : 0.5776, test rmse : 0.6023, # ensemble : 200
8th sample train rmse : 0.5731, test rmse : 0.5973, # ensemble : 200
9th sample train rmse : 0.5693, test rmse : 0.5939, # ensemble : 200


In [25]:
predict_output = main.predict(new_data = test_x,config=config, model = samples)
test_rmse = np.sqrt( np.mean( (np.array(test_y.flatten()) - predict_output)**2 ) )
print(f"Test RMSE : {test_rmse}")

Test RMSE : 0.6186897804924161
